Done by: Nicholas Baron (012542621) and Kyle Ah-Tye (012028367)

In this assignment, you will familiarize yourself with:

*   Python
*   Jupyter Notebooks
*   Google Colab

to develop a n-gram based language model, with smoothing to be able to

* Produce naturalish text
* Rate the perplexity of a text given your model

First, we will load in some data.

In [ ]:
!wget -O corpus.zip https://digitalrepository.wheatoncollege.edu/bitstream/handle/11040/24451/Shakespeare%20Corpus%20by%20Play.zip?sequence=1&isAllowed=y 

--2021-02-09 23:57:37--  https://digitalrepository.wheatoncollege.edu/bitstream/handle/11040/24451/Shakespeare%20Corpus%20by%20Play.zip?sequence=1
Resolving digitalrepository.wheatoncollege.edu (digitalrepository.wheatoncollege.edu)... 52.45.246.5
Connecting to digitalrepository.wheatoncollege.edu (digitalrepository.wheatoncollege.edu)|52.45.246.5|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8536913 (8.1M) [application/octet-stream]
Saving to: ‘corpus.zip’

corpus.zip          100%[===================>]   8.14M  --.-KB/s    in 0.1s    

2021-02-09 23:57:37 (54.8 MB/s) - ‘corpus.zip’ saved [8536913/8536913]



By typing "!" in a notebook, you can use command line prompts

In [ ]:
!ls 

 corpus.zip   sample_data  'Shakespeare Corpus by Play'



Now, we will unzip the corpus


In [ ]:
!rm -rvf Shake* 
!unzip corpus.zip

We can even have python and the command line interact.

In [ ]:
files = !ls Shake*/*/*_noSCRB.txt 

`files` is now a list containing all of the files we wish to use in our data

In [ ]:
print(files)
print(len(files))

["'Shakespeare Corpus by Play/allsWellThatEndsWell/allsWellThatEndsWell_noSCRB.txt'", "'Shakespeare Corpus by Play/antonyAndCleopatra/antonyAndCleopatra_noSCRB.txt'", "'Shakespeare Corpus by Play/asYouLikeIt/asYouLikeIt_noSCRB.txt'", "'Shakespeare Corpus by Play/comedyOfErrors/comedyOfErrors_noSCRB.txt'", "'Shakespeare Corpus by Play/coriolanus/coriolanus_noSCRB.txt'", "'Shakespeare Corpus by Play/cymbeline/cymbeline_noSCRB.txt'", "'Shakespeare Corpus by Play/edwardIII/edwardIII_noSCRB.txt'", "'Shakespeare Corpus by Play/hamlet/hamlet_noSCRB.txt'", "'Shakespeare Corpus by Play/henryIVparti/henryIVparti_noSCRB.txt'", "'Shakespeare Corpus by Play/henryIVpartii/henryIVpartii_noSCRB.txt'", "'Shakespeare Corpus by Play/henryV/henryV_noSCRB.txt'", "'Shakespeare Corpus by Play/henryVIII/henryVIII_noSCRB.txt'", "'Shakespeare Corpus by Play/henryVIparti/henryVIparti_noSCRB.txt'", "'Shakespeare Corpus by Play/henryVIpartii/henryVIpartii_noSCRB.txt'", "'Shakespeare Corpus by Play/henryVIpartiii/h

Note, there are extra quotes on the files, so we will use list comprehension to remove those

In [ ]:
files = [file[1:-1] for file in files]

#STEP 1


* Go through all of the files
  * Read them in
  * Iterate through them line by line
  * Split each line by white space
  * Make sure to add in a special newline token `'\n'` for every line
  * Every file should be in its own list
  * Add special start `'<s>'` and end `'</s>'` tokens to the list for each file
  * The end results should be an list of lists of strings called `texts`



  e.g.

  If we read in one file that looked like:
  
    HAMLET: Alas poor Yorick.

    GRAVEDIGGER: Put that skull down.



  The end results should look like:

  `texts = [['<s>', 'HAMLET:', 'Alas', 'poor', 'Yorick.','\n','GRAVEDIGGER:', Put','that', 'skull', 'down.', '</s>']]`
 

In [ ]:
texts = []

## STEP 1 -- 
for fileName in files:
  with open(fileName,'r') as plot_file:
    textIter = ['<s>']
    for line in plot_file:
      lstLine = line.rstrip().split()
      textIter += lstLine
      textIter.append('\n')
  
    textIter.append('</s>')
    texts.append(textIter)


#STEP 2

With our files read in, it's time to do some counting!

* Implement the function `get_counts(texts,n)` which takes in a list of list of strings -- `texts` -- and the arity of the n-gram -- `n` 
* `get_counts` should return a dictionary where the keys are the n-grams (as tuples of strings) and the values are the integer counts of all of the found n-grams


Here's a quick primer on tuples in Python

    triple = (1,2,3)
    double = (1,2)
    single = (1,) #note the trailing comma to distinguish from parenethesis 
    null_tuple = tuple() 

    list_triple = [1,2,3]
    tuple_triple = tuple(list_triple)
    


In [ ]:
def get_counts(texts,n):
  ngrams = {}
  for words in texts:
    #unigrams
    if n == 1:
      for word in words:
        word = (word,)
        ngrams[word] = ngrams.get(word,0)+1
  
    #bigrams
    elif n == 2:
      for pair in zip(words,words[1:]):
        ngrams[pair] = ngrams.get(pair,0)+1

    #trigrams
    elif n == 3:
      for pair in zip(words,words[1:],words[2:]):
        ngrams[pair] = ngrams.get(pair,0)+1

  return ngrams

unigrams = get_counts(texts,1)
bigrams = get_counts(texts,2)
trigrams = get_counts(texts,3)


#STEP 3

With our counts, it's time to make our first language model.

* Implement the class NGramLM 
  * Initialize the LM
    * First use `get_counts` to get the raw counts of the n-grams up to order `n`
    * Convert these raw counts into probabilities
      * ### Question 1:  What is the relationship between raw counts of n-grams of order n and n-1 and the probabilities of the next word for order n? 
      * P(w|w1, w2, ...) = P(w1, w2, ... w) / P(w1, w2, ...)
  * Implement the `generate_text` function
    * You are given an optional prompt (a list of strings)
    * You should return a list of strings based on randomly choosing according to the probabilities of the language model (using the `random` library)
    * Your function should fail gracefully if an n-gram can't be found by backing off
  * Implement `score_text` which when given a text (a list of strings) returns the perplexity of that text
    * Make sure you don't have any divide by 0 (or log(0)) issues -- return float('inf') if there is an n-gram not found in the model
    * Use backoff to try to somewhat gracefully handle issues

In [ ]:
import random, math
random.seed(0)

class NGramLM():
  def __init__(self,texts,n):
    self.n = n

    trigram_counts = get_counts(texts,3)

    total_trigrams = sum(trigram_counts.values())

    self.trigramProbabilities = {
        gram: trigram_counts[gram] / total_trigrams
        for gram, _ in sorted(trigram_counts.items(), key= lambda x: (x[1], x[0]), reverse=True)
    }

    bigram_counts = get_counts(texts,2)
    total_bigrams = sum(bigram_counts.values())

    self.bigramProbabilities = {
        gram: bigram_counts[gram] / total_bigrams
        for gram, _ in sorted(bigram_counts.items(), key= lambda x: (x[1], x[0]), reverse=True)
    }

    unigram_counts = get_counts(texts,1)
    total_unigrams = sum(unigram_counts.values())

    self.unigramProbabilities = {
        gram: unigram_counts[gram] / total_unigrams
        for gram, _ in sorted(unigram_counts.items(), key= lambda x: (x[1], x[0]), reverse=True)
    }

    # length = count of additional words to append to the prompt
  def generate_text(self, length,prompt = []):
    result = ['<s>'] if len(prompt) == 0 else prompt

    for _ in range(length):
        ngram_lvl = self.n
        added_word = False

        while not added_word:
            added_word = False
            if ngram_lvl == 1:
                choosen_unigram = random.choices(*zip(*self.unigramProbabilities.items()))[0]
                result.append(choosen_unigram[0])
                added_word = True
            elif ngram_lvl == 2 and len(result) != 0:
                last_words = result[-1]
                possible_bigrams = {
                    gram: value for gram, value in self.bigramProbabilities.items() if gram[0] == last_words
                }
                if len(possible_bigrams) == 0:
                    ngram_lvl -= 1
                    continue
                choosen_bigram = random.choices(*zip(*possible_bigrams.items()))[0]
                result.append(choosen_bigram[1])
                added_word = True
            elif ngram_lvl == 3 and len(result) >= 2:
                prev_last_word, last_word = result[-2], result[-1]
                possible_trigrams = {
                    gram: value for gram, value in self.trigramProbabilities.items() if gram[0] == prev_last_word and gram[1] == last_word
                }
                if len(possible_trigrams) == 0:
                    ngram_lvl -= 1
                    continue
                choosen_trigram = random.choices(*zip(*possible_trigrams.items()))[0]
                result.append(choosen_trigram[2]) 
                added_word = True
            else:
                ngram_lvl -= 1

    return result

  def score_text(self,text):
    num_samples = len(text)
    current_gram = []
    score = 0

    for word in text:
        # Get the P(word | prev)
        current_gram.append(word)
        if len(current_gram) > self.n:
            current_gram.pop(0)

        prob_current_gram = None
        prob_prev_gram = None
        for lvl in range(self.n):
            tuple_gram = tuple(current_gram[lvl:])
            prev = tuple(current_gram[lvl:-1])
            if len(tuple_gram) == 3:
                if tuple_gram in self.trigramProbabilities and prev in self.bigramProbabilities:
                    prob_current_gram = self.trigramProbabilities[tuple_gram]
                    prob_prev_gram = self.bigramProbabilities[prev]
                    break
            elif len(tuple_gram) == 2:
                if tuple_gram in self.bigramProbabilities and prev in self.unigramProbabilities:
                    prob_current_gram = self.bigramProbabilities[tuple_gram]
                    prob_prev_gram = self.unigramProbabilities[prev]
                    break
            elif len(tuple_gram) == 1:
                if tuple_gram in self.unigramProbabilities:
                    prob_current_gram = self.unigramProbabilities[tuple_gram]
                    prob_prev_gram = 0
                    break

        if prob_current_gram is None:
            return float('inf')

        prob_word = prob_current_gram / prob_prev_gram if prob_prev_gram != 0 \
            else prob_current_gram

        if prob_word == 0:
            return float('inf')
        else:
            score += math.log2(prob_word)
    
    return math.pow(2, score * -1 / num_samples)


In [ ]:
trigramLM = NGramLM(texts,3)

print(' '.join(trigramLM.generate_text(100)))
print(' '.join(trigramLM.generate_text(100,['HAMLET.'])))
print(' '.join(trigramLM.generate_text(100,['palm', 'to', 'palm', 'is', 'holy', 'palmers', 'kiss'])))
print(' '.join(trigramLM.generate_text(100,['Hamlet','just','use','Google'])))


print(trigramLM.score_text(['IBM','announced','today','that','they','will','be','buying','Google']))
print(trigramLM.score_text(['palm', 'to', 'palm', 'is', 'holy', "palmers'", 'kiss.']))
print(trigramLM.score_text(['what','do','you','say','to','that','Hamlet']))

<s> 1612 
 
 Enter his chamber, as it would do it! 
 What private griefs they have, 
 Is foul as slander. 
 OTHELLO. That which upholdeth him that beateth it! 
 Why, hath your Highness should intend this voyage 
 Is jollity for apes and monkeys, 
 As ample as myself. 
 They call drinking deep, dying scarlet; and when they behold a greater soldier than he touch'd mine. 
 TRANIO. That's but a 
 fellow! Go thy ways. 
 FALSTAFF. God save you! 
 DULL. And I shall conduct you, 
 Here 'tis; commends it to me. 

HAMLET. Take their examination yourself and all that's in't is precious. If you will swear 
 A mother, and my misgiving still 
 dwell in vegetives, in metals, stones; 
 Who makes that noise 
 there? 
 Fool. O nuncle, court holy water on the old time Shall lead him to some alive, 
 I lay unto the grave. 
 YORK. Which now lies foul and muddy. Not one to the graver 
 purpose, confess thyself- 
 This world is almost mature 
 A plot upon her knees than on the lute, and you must call me fath

#STEP 4

Build a better language model

* Implement the function `truncate_texts` which takes in texts and a threshold
  * Replace all words with counts less than the threshold with `'<UNK>'`
* Implement the function `get_vocabulary` which takes in texts and returns a set of all of the words found in the texts
* Implement the LaplaceLM language model
  * Use add-alpha Laplace smoothing to calculate the probabilities
  * Implement the `generate_text` and `score_text` functions as above EXCEPT
    * Handle unknown vocabulary gracefully by checking the vocabulary and replacing with `'<UNK>'`

In [ ]:
def truncate_texts(texts,threshold):
  modified_texts = []

  word_counts = {}
  for text in texts:
      for word in text:
          word_counts[word] = word_counts.get(word, 0) + 1
  
  for text in texts:
      modified_texts.append([word if word_counts[word] > threshold else '<UNK>' for word in text])

  return modified_texts

def get_vocabulary(texts):
  vocab = set()

  for text in texts:
      vocab |= set(text)

  return vocab

class LaplaceLM(NGramLM):
  def __init__(self,texts,n,vocabulary, alpha=1):
    self.n = n
    self.vocab = vocabulary

    trigram_counts = get_counts(texts,3)
    total_trigrams = sum(trigram_counts.values())

    self.trigramProbabilities = {
        gram: (trigram_counts[gram]+alpha) / total_trigrams
        for gram, _ in sorted(trigram_counts.items(), key= lambda x: (x[1], x[0]), reverse=True)
    }

    bigram_counts = get_counts(texts,2)
    total_bigrams = sum(bigram_counts.values())

    self.bigramProbabilities = {
        gram: (bigram_counts[gram]+alpha) / total_bigrams 
        for gram, _ in sorted(bigram_counts.items(), key= lambda x: (x[1], x[0]), reverse=True)
    }

    unigram_counts = get_counts(texts,1)
    total_unigrams = sum(unigram_counts.values())

    self.unigramProbabilities = {
        gram: (unigram_counts[gram]+alpha) / (total_unigrams + len(vocab))
        for gram, _ in sorted(unigram_counts.items(), key= lambda x: (x[1], x[0]), reverse=True)
    }

 
  def generate_text(self, length,prompt=[]):
    # replace words not found in vocab with 'unk'
    result = ['<s>'] if len(prompt) == 0 \
        else [ 
            word if word in self.vocab else '<UNK>' for word in prompt
        ]

    for _ in range(length):
        ngram_lvl = self.n
        added_word = False

        while not added_word:
            added_word = False
            if ngram_lvl == 1:
                choosen_unigram = random.choices(*zip(*self.unigramProbabilities.items()))[0]
                result.append(choosen_unigram[0])
                added_word = True
            elif ngram_lvl == 2 and len(result) != 0:
                last_words = result[-1]
                possible_bigrams = {
                    gram: value for gram, value in self.bigramProbabilities.items() if gram[0] == last_words
                }
                if len(possible_bigrams) == 0:
                    ngram_lvl -= 1
                    continue
                choosen_bigram = random.choices(*zip(*possible_bigrams.items()))[0]
                result.append(choosen_bigram[1])
                added_word = True
            elif ngram_lvl == 3 and len(result) >= 2:
                prev_last_word, last_word = result[-2], result[-1]
                possible_trigrams = {
                    gram: value for gram, value in self.trigramProbabilities.items() if gram[0] == prev_last_word and gram[1] == last_word
                }
                if len(possible_trigrams) == 0:
                    ngram_lvl -= 1
                    continue
                choosen_trigram = random.choices(*zip(*possible_trigrams.items()))[0]
                result.append(choosen_trigram[2]) 
                added_word = True
            else:
                ngram_lvl -= 1

    return result

  def score_text(self,text):
    num_samples = len(text)
    current_gram = []
    score = 0

    for word in text:
        # Get the P(word | prev)
        current_gram.append(word if word in self.vocab else '<UNK>')
        if len(current_gram) > self.n:
            current_gram.pop(0)

        prob_current_gram = None
        prob_prev_gram = None
        for lvl in range(self.n):
            tuple_gram = tuple(current_gram[lvl:])
            prev = tuple(current_gram[lvl:-1])
            if len(tuple_gram) == 3:
                if tuple_gram in self.trigramProbabilities and prev in self.bigramProbabilities:
                    prob_current_gram = self.trigramProbabilities[tuple_gram]
                    prob_prev_gram = self.bigramProbabilities[prev]
                    break
            elif len(tuple_gram) == 2:
                if tuple_gram in self.bigramProbabilities and prev in self.unigramProbabilities:
                    prob_current_gram = self.bigramProbabilities[tuple_gram]
                    prob_prev_gram = self.unigramProbabilities[prev]
                    break
            elif len(tuple_gram) == 1:
                if tuple_gram in self.unigramProbabilities:
                    prob_current_gram = self.unigramProbabilities[tuple_gram]
                    prob_prev_gram = 0
                    break

        if prob_current_gram is None:
            return float('inf')

        prob_word = prob_current_gram / prob_prev_gram if prob_prev_gram != 0 \
            else prob_current_gram

        if prob_word == 0:
            return float('inf')
        else:
            score += math.log2(prob_word)
    
    return math.pow(2, score * -1 / num_samples)


truncated = truncate_texts(texts,5)
vocab = get_vocabulary(truncated)

for alpha in [0,1e-5,0.1,1,10,1000]:
    print(f'LM with alpha of {alpha}')
    trigramLM = LaplaceLM(truncated,3,vocab,alpha)

    print(' '.join(trigramLM.generate_text(100)))
    print(' '.join(trigramLM.generate_text(100,['HAMLET.'])))
    print(' '.join(trigramLM.generate_text(100,['palm', 'to', 'palm', 'is', 'holy', 'palmers', 'kiss'])))
    print(' '.join(trigramLM.generate_text(100,['Hamlet','just','use','Google'])))

    print(trigramLM.score_text(['IBM','announced','today','that','they','will','be','buying','Google']))
    print(trigramLM.score_text(['palm', 'to', 'palm', 'is', 'holy', 'palmers', 'kiss']))
    print(trigramLM.score_text(['what','do','you','say','to','that','Hamlet']))

LM with alpha of 0
<s> <UNK> 
 
 
 Re-enter PROVOST 
 CLAUDIO. Call it my fault to heaven, 
 <UNK> farewell! for wisdom cries out 'No <UNK> <UNK> 
 Sword, I hold you still; 
 And <UNK> I could not stay a jot longer. 
 Bene. Do you note us. 
 Most spend their mouths if they came to serve all hopes <UNK> 
 BOTTOM. Well, I am satisfied. Give me ink and paper, 
 Found that the <UNK> night by <UNK> this our watch, so like the commandment of a <UNK> would say, <UNK> or, for thy father's care; 
 And I have
<UNK> 
 If you bethink you of fourscore pounds a year and a <UNK> <UNK> 
 Upon these <UNK> 
 If either of 
 my part, I breathe again 
 That God, the best of gold 'gainst this which he lost sixpence a day 
 That <UNK> the rest o' th' <UNK> <UNK> that <UNK> of a calm world and wild and wand'ring flood; 
 When birds do sing. 
 Exeunt fighting 
 
 THALIARD. 
 The poor advanc'd makes friends of them, 
 And sent me to a maid she <UNK> 
 Nurse. Faith, I perceive you delight not in
palm to palm is 

### Question 2: What do you notice with the changing of the alpha?  How do perplexities change?  How does the quality of the generated text change? How do they differ from the simple LM?
The perplexities get lower as alpha increases. The number of unknowns seems to decrease with higher values of alpha. However, by our judgement, no one text is better than another, including the NGram model.


Bonus time:

#### 2 points:  Find 2 text datasets (they can be of your own creation, pre-existing datasets, text you scrape, etc.) and combine them together to make an amusing bot.  

e.g. look at https://kingjamesprogramming.tumblr.com/ which combines the bible with Structure and Interpretation of Computer Programs

Showcase your best efforts below

#### 8 points: Make a LM like above except use Kneser-Ney smoothing.  Show it's results and discuss how they differ from the above models


**Star wars episode IV, V, and VII dialogue**

In [ ]:
!wget -O star_wars_iv https://raw.githubusercontent.com/kamran786/Star-Wars-Movie-Scripts-Text-Analysis/master/SW_EpisodeIV.txt

--2021-02-10 00:09:21--  https://raw.githubusercontent.com/kamran786/Star-Wars-Movie-Scripts-Text-Analysis/master/SW_EpisodeIV.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 78278 (76K) [text/plain]
Saving to: ‘star_wars_iv’

star_wars_iv        100%[===================>]  76.44K  --.-KB/s    in 0.02s   

2021-02-10 00:09:21 (4.97 MB/s) - ‘star_wars_iv’ saved [78278/78278]



In [ ]:
!wget -O star_wars_v https://raw.githubusercontent.com/kamran786/Star-Wars-Movie-Scripts-Text-Analysis/master/SW_EpisodeV.txt

--2021-02-10 00:09:21--  https://raw.githubusercontent.com/kamran786/Star-Wars-Movie-Scripts-Text-Analysis/master/SW_EpisodeV.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 55487 (54K) [text/plain]
Saving to: ‘star_wars_v’

star_wars_v         100%[===================>]  54.19K  --.-KB/s    in 0.01s   

2021-02-10 00:09:21 (4.29 MB/s) - ‘star_wars_v’ saved [55487/55487]



In [ ]:
!wget -O star_wars_vi https://raw.githubusercontent.com/kamran786/Star-Wars-Movie-Scripts-Text-Analysis/master/SW_EpisodeVI.txt

--2021-02-10 00:09:21--  https://raw.githubusercontent.com/kamran786/Star-Wars-Movie-Scripts-Text-Analysis/master/SW_EpisodeVI.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 48176 (47K) [text/plain]
Saving to: ‘star_wars_vi’

star_wars_vi        100%[===================>]  47.05K  --.-KB/s    in 0.01s   

2021-02-10 00:09:22 (3.78 MB/s) - ‘star_wars_vi’ saved [48176/48176]



In [ ]:
#files = ['star_wars_iv', 'star_wars_v']
files = ['star_wars_iv', 'star_wars_v', 'star_wars_vi']

In [ ]:
star_wars_words = []
for fileName in files:
  with open(fileName) as fobj:
      temp_words = []
      for line in fobj:
          temp_words.extend(line.split())
          temp_words.append('\n')
      star_wars_words.append(temp_words)



In [ ]:
trigramLM = NGramLM(star_wars_words,3)

print(' '.join(trigramLM.generate_text(1000)))
print(' '.join(trigramLM.generate_text(1000,['There','will','be','no','bargain'])))
print(trigramLM.score_text(['There','will','be','no','bargain']))

<s> "LUKE" "Easy, Chewie." 
 "802" "LEIA" "Lando?" 
 "803" "DODONNA" "The battle station plans are not in the outer defense." 
 "804" "GOLD LEADER" "I copy, Gold Five." 
 "922" "GOLD FIVE" "Stabilize your rear deflectors. Watch for enemy fighters." 
 "870" "RED LEADER" "I get no reading. Are you kidding! How?" 
 "483" "BEN" "Destroyed... by the Empire!" 
 "484" "YODA" "You will take quite awhile to evacuate the T-forty- sevens." 
 "151" "LUKE" "Well, wait a minute. Where'd she go? Bring her back! Play back the entire Imperial computer network." 
 "538" "GREEN LEADER" "Green Leader standing by." 
 "843" "RED SEVEN" "Red Seven standing by." 
 "847" "WEDGE" "Red Three, standing by." 
 "538" "GREEN LEADER" "Green Leader standing by." 
 "847" "WEDGE" "Red Three, standing by." 
 "539" "THREEPIO" "The city's central computer told you? Artoo-Detoo, you know who he's talking about?" 
 "161" "HAN" "Whoa! Whoa! Help!" 
 "153" "THREEPIO" "I'm sorry, too." 
 "676" "THREEPIO" "They're madmen! They'r

In [ ]:
truncated = truncate_texts(star_wars_words,2)
vocab = get_vocabulary(truncated)
for alpha in [0,1e-5,0.1,1,10,1000]:
  print(f'LM with alpha of {alpha}')
  trigramLM = LaplaceLM(truncated,3,vocab,alpha)

  print(' '.join(trigramLM.generate_text(100)))

LM with alpha of 0
<s> not the droids would now be in the south <UNK> <UNK> 
 <UNK> "RED LEADER" "I copy, Gold Leader. Move into position." 
 <UNK> "HAN" "What's going on?" 
 "561" "HAN" "No. But he has in mind." 
 "27" "LEIA" "Han, we need you!" 
 "28" <UNK> "It was just going <UNK> Turn around, Chewbacca, I can't see, pal. What's goin' on?" 
 "430" "CREATURE" "Not far. Yoda not far. <UNK> Soon I'll be up there with you! Listen, have I got him!" 
 <UNK> "DODONNA" <UNK> your <UNK> This way is much <UNK> than a dark <UNK> then you
LM with alpha of 1e-05
<s> you will be. You will never find a way back into the detention <UNK> 
 "370" <UNK> "The Emperor will <UNK> 
 "362" "BEN" "You'll have to face <UNK> that I could take on the <UNK> If the Rebels against this station would be the power of the ship, make sure there aren't any <UNK> <UNK> 
 "299" "LEIA" <UNK> 
 "503" "HAN" "Then I am See-Threepio, <UNK> <UNK> will you? I want to learn what seems to be <UNK> <UNK> of some <UNK> I wonder <U